TODO Import the necessary libraries

In [16]:
import numpy as np
import pandas as pd
import datetime as dt

TODO Import the dataset 

In [277]:
data = pd.read_csv('./data/weather_dataset.data', sep = "\s+")

In [278]:
data.head()

,Yr,Mo,Dy,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12
0,61,1,1,15.04,14.96,13.17,9.29,NaN,9.87,13.67,10.25,10.83,12.58,18.50,15.04
1,61,1,2,14.71,NaN,10.83,6.50,12.62,7.67,11.50,10.04,9.79,9.67,17.54,13.83
2,61,1,3,18.50,16.88,12.33,10.13,11.17,6.17,11.25,NaN,8.50,7.67,12.75,12.71
3,61,1,4,10.58,6.63,11.75,4.58,4.54,2.88,8.63,1.79,5.83,5.88,5.46,10.88
4,61,1,5,13.33,13.25,11.42,6.17,10.71,8.21,11.92,6.54,10.92,10.34,12.92,11.83


TODO  Assign it to a variable called data and replace the first 3 columns by a proper datetime index

In [279]:
data["Date"] = pd.to_datetime(data[["Yr","Mo","Dy"]].astype(str).agg('-'.join, axis=1))

In [280]:
data = data.drop(columns=["Yr","Mo","Dy"])

TODO Write a function in order to fix date (this relate only to the year info) and apply it

In [281]:
def make_year(date):
    if date.year > 2000:
        year = date.year - 100 
    else:
        year = date.year
    return dt.date(year, date.month, date.day)

data['Date'] = data['Date'].apply(make_year)

TODO Check if everything is okay with the data. Create functions to delete/fix rows with strange cases and apply them

In [284]:
for loc in data.columns[:-1]:
    data[loc] = data[loc].str.replace(',', '.').astype(str)

In [285]:
#throw out negative values of wind speed 
data = data.loc[(data['loc9'] > 0) & (data['loc11'] > 0)]

TODO Set the right dates as the index. Pay attention at the data type, it should be datetime64[ns]

In [286]:
data = data.set_index("Date")

In [287]:
data.index = data.index.astype("datetime64[ns]")

Check again if everything is okay with the data.

In [311]:
#after set the right dates as the index we will return to the wrong data problem (invalid values will be set as NaN):
data = data.apply(pd.to_numeric, errors='coerce').dropna()

In [314]:
data.head()

,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12
Date,,,,,,,,,,,,
1961-01-04,10.58,6.63,11.75,4.58,4.54,2.88,8.63,1.79,5.83,5.88,5.46,10.88
1961-01-05,13.33,13.25,11.42,6.17,10.71,8.21,11.92,6.54,10.92,10.34,12.92,11.83
1961-01-06,13.21,8.12,9.96,6.67,5.37,4.50,10.67,4.42,7.17,7.50,8.12,13.17
1961-01-07,13.50,14.29,9.50,4.96,12.29,8.33,9.17,9.29,7.58,7.96,13.96,13.79
1961-01-08,10.96,9.75,7.62,5.91,9.62,7.29,14.29,7.62,9.25,10.46,16.62,16.46


TODO Compute how many values are missing for each location over the entire record

In [103]:
data.isnull().sum()

loc1     6
loc2     3
loc3     3
loc4     5
loc5     2
loc6     0
loc7     3
loc8     2
loc9     4
loc10    1
loc11    0
loc12    4
dtype: int64

TODO Compute how many non-missing values there are in total

In [105]:
data.notnull().sum()

loc1     6568
loc2     6571
loc3     6571
loc4     6569
loc5     6572
loc6     6574
loc7     6571
loc8     6572
loc9     6570
loc10    6573
loc11    6574
loc12    6570
dtype: int64

TODO Calculate the mean windspeeds of the windspeeds over all the locations and all the times

In [244]:
data.sum().sum() / data.notnull().sum().sum()

10.224252274344378

TODO Create a DataFrame called loc_stats and calculate the min, max and mean windspeeds 
and standard deviations of the windspeeds at each location over all the days

In [209]:
loc_stats = pd.DataFrame()

In [245]:
loc_stats['min'] = data.min(axis = 0) # min
loc_stats['max'] = data.max(axis = 0) # max 
loc_stats['mean'] = data.mean(axis = 0) # mean
loc_stats['std'] = data.std(axis = 0) # standard deviations

TODO Find the average windspeed in January for each location

In [289]:
data.loc[data.index.month == 1].mean()

loc1     14.847325
loc2     12.914560
loc3     13.299624
loc4      7.199498
loc5     11.667734
loc6      8.054839
loc7     11.819355
loc8      9.512047
loc9      9.543208
loc10    10.053566
loc11    14.550520
loc12    18.028763
dtype: float64

TODO Downsample the record to a yearly frequency for each location

In [291]:
data.groupby(data.index.to_period('y')).mean()

,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12
Date,,,,,,,,,,,,
1961,12.219831,10.283212,11.264218,6.889216,10.819081,7.679169,9.691620,8.787103,8.642964e+00,9.776194,13.410055,13.625850
1962,12.246923,10.110438,11.732712,6.960440,10.657918,7.393068,11.020712,8.793753,8.316822e+00,9.676247,12.930685,14.323956
1963,12.813452,10.836986,12.541151,7.330055,11.724110,8.434712,11.075699,10.336548,8.903589e+00,10.224438,13.638877,14.999014
1964,12.363661,10.920164,12.104372,6.787787,11.454481,7.570874,10.259153,9.467350,7.789016e+00,10.207951,13.740546,14.910301
1965,12.451370,11.075534,11.848767,6.858466,11.024795,7.478110,10.618712,8.879918,7.907425e+00,9.918082,12.964247,15.591644
1966,13.461973,11.557205,12.020630,7.345726,11.805041,7.793671,10.579808,8.835096,8.514438e+00,9.768959,14.265836,16.307260
1967,12.737151,10.990986,11.739397,7.143425,11.630740,7.368164,10.652027,9.325616,8.645014e+00,9.547425,14.774548,17.135945
1968,11.835628,10.468197,11.409754,6.477678,10.760765,6.067322,8.859180,8.255519,7.224945e+00,7.832978,12.808634,15.017486
1969,11.166356,9.723699,10.902000,5.767973,9.873918,6.189973,8.564493,7.711397,7.924521e+00,7.754384,12.621233,15.762904


TODO Downsample the record to a monthly frequency for each location

In [292]:
data.groupby(data.index.to_period('m')).mean()

,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12
Date,,,,,,,,,,,,
1961-01,14.841333,11.988333,13.431613,7.736774,11.072759,8.588065,11.184839,9.245333,9.085806,10.107419,13.880968,14.703226
1961-02,16.269286,14.975357,14.250385,9.230741,13.852143,10.937500,11.890714,11.846071,11.821429,12.714286,18.583214,15.411786
1961-03,10.581429,11.177241,10.681379,7.213214,10.342069,8.693793,9.445862,9.534828,10.294138,11.142759,16.291379,15.807931
1961-04,10.722667,9.427667,9.998000,5.830667,8.435000,6.495000,6.925333,7.094667,7.342333,7.237000,11.147333,10.278333
1961-05,9.860968,8.850000,10.818065,5.905333,9.490323,6.574839,7.604000,8.177097,8.039355,8.499355,11.900323,12.011613
...,...,...,...,...,...,...,...,...,...,...,...,...
1978-08,9.645161,8.259355,9.032258,4.502903,7.368065,5.935161,5.650323,5.417742,7.241290,5.536774,10.466774,12.054194
1978-09,10.913667,10.895000,10.635000,5.725000,10.372000,9.278333,10.790333,9.583000,10.069333,8.939000,15.680333,19.391333
1978-10,9.897742,8.670968,9.295806,4.721290,8.525161,6.774194,8.115484,7.337742,8.297742,8.243871,13.776774,17.150000


TODO Downsample the record to a weekly frequency for each location

In [294]:
data.groupby(data.index.to_period('W')).mean()

,loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10,loc11,loc12
Date,,,,,,,,,,,,
1960-12-26/1961-01-01,15.040000,14.960000,13.170000,9.290000,NaN,9.870000,13.670000,10.250000,10.830000,12.580000,18.500000,15.040000
1961-01-02/1961-01-08,13.541429,11.486667,10.487143,6.417143,9.474286,6.435714,11.061429,6.616667,8.434286,8.497143,12.481429,13.238571
1961-01-09/1961-01-15,12.468571,8.967143,11.958571,4.630000,7.351429,5.072857,7.535714,6.820000,5.712857,7.571429,11.125714,11.024286
1961-01-16/1961-01-22,13.204286,9.862857,12.982857,6.328571,8.966667,7.417143,9.257143,7.875714,7.145714,8.124286,9.821429,11.434286
1961-01-23/1961-01-29,19.880000,16.141429,18.225714,12.720000,17.432857,14.828571,15.528571,15.160000,14.480000,15.640000,20.930000,22.530000
...,...,...,...,...,...,...,...,...,...,...,...,...
1978-11-27/1978-12-03,14.934286,11.232857,13.941429,5.565714,10.215714,8.618571,9.642857,7.685714,9.011429,9.547143,11.835714,18.728571
1978-12-04/1978-12-10,20.740000,19.190000,17.034286,9.777143,15.287143,12.774286,14.437143,12.488571,13.870000,14.082857,18.517143,23.061429
1978-12-11/1978-12-17,16.758571,14.692857,14.987143,6.917143,11.397143,7.272857,10.208571,7.967143,9.168571,8.565714,11.102857,15.562857


TODO Calculate the min, max and mean windspeeds and standard deviations of the windspeeds across all locations for each week 
(assume that the first week starts on January 2 1961) for the first 21 weeks

In [304]:
period = data[(data.index.year == 1961)]

In [307]:
period.resample('W').agg(['min','max','mean','std']).head(21)

loc1                               loc2                    \
              min    max       mean       std    min    max       mean   
Date                                                                     
1961-01-01  15.04  15.04  15.040000       NaN  14.96  14.96  14.960000   
1961-01-08  10.58  18.50  13.541429  2.631321   6.63  16.88  11.486667   
1961-01-15   9.04  19.75  12.468571  3.555392   3.54  12.08   8.967143   
1961-01-22   4.92  19.83  13.204286  5.337402   3.42  14.37   9.862857   
1961-01-29  13.62  25.04  19.880000  4.619061   9.96  23.91  16.141429   
1961-02-05  10.58  24.21  16.827143  5.251408   9.46  24.21  15.460000   
1961-02-12  16.00  24.54  19.684286  3.587677  11.54  21.42  16.417143   
1961-02-19   6.04  22.50  15.130000  5.064609  11.63  20.17  15.091429   
1961-02-26   7.79  25.80  15.221429  7.020716   7.08  21.50  13.625714   
1961-03-05  10.96  13.33  12.006667  1.057878   8.83  17.00  12.921667   
1961-03-12   4.88  14.79   9.376667  3.732263   8.08  16.96  11.578571   
1961-03-19   4.92  16.88  11.911429  3.860036   9.46  15.54  13.501429   
1961-03-26   6.29  15.00   9.567143  3.613298   2.58  11.63   8.387143   
1961-04-02   5.88  18.25   9.591667  4.376722   3.50  16.29   8.175000   
1961-04-09   4.50  18.12  11.964286  4.604392   7.04  14.62  10.654286   
1961-04-16   4.71  15.50   8.965714  3.937727   4.83  12.25   8.000000   
1961-04-23   4.00  21.09  12.621429  5.676655   3.71  15.41  10.438571   
1961-04-30   4.08  16.29  10.117143  4.349662   6.50  14.46   9.798571   
1961-05-07   9.87  23.00  15.367143  5.025507  10.29  19.79  13.970000   
1961-05-14   3.54  12.79   7.772857  3.371022   3.96  15.12   8.712857   
1961-05-21   4.88  15.04   8.225714  3.631730   3.58  10.17   5.631667   

                       loc3         ...      loc10            loc11         \
                 std    min    max  ...       mean       std    min    max   
Date                                ...                                      
1961-01-01       NaN  13.17  13.17  ...  12.580000       NaN  18.50  18.50   
1961-01-08  3.949525   7.62  12.33  ...   8.497143  1.704941   5.46  17.54   
1961-01-15  3.148945   7.08  19.50  ...   7.571429  4.084293   5.25  20.71   
1961-01-22  3.837785   7.29  20.79  ...   8.124286  4.783952   6.50  15.92   
1961-01-29  5.170224  12.67  25.84  ...  15.640000  3.713368  14.04  27.71   
1961-02-05  5.187395   9.04  19.70  ...   9.460000  2.839501   9.17  19.33   
1961-02-12  3.608373  13.67  21.34  ...  14.440000  1.746749  15.21  26.38   
1961-02-19  3.575012   6.13  17.25  ...  13.542857  2.531361  14.09  29.63   
1961-02-26  5.147348   6.08  22.42  ...  12.730000  4.920064   9.59  23.21   
1961-03-05  3.122969   8.17  13.67  ...  12.396667  1.744083  11.58  23.45   
1961-03-12  3.230167   7.54  16.38  ...  10.458571  3.655113  10.21  22.71   
1961-03-19  2.352867   5.25  13.96  ...  11.627143  3.099472  11.29  22.79   
1961-03-26  3.657265   4.79  15.63  ...  11.481429  2.538224   8.25  21.34   
1961-04-02  4.743985   5.09  14.96  ...   8.993333  2.966423   7.21  18.63   
1961-04-09  2.845399   9.29  18.29  ...   7.238571  2.336182   7.62  17.16   
1961-04-16  2.607118   3.92  15.79  ...   6.178571  2.161137   5.75  16.17   
1961-04-23  4.631736   3.33  17.00  ...   9.551429  3.347972   6.75  19.21   
1961-04-30  2.871425   2.54  14.96  ...   6.124286  2.840568   5.13  13.04   
1961-05-07  3.750835   8.42  21.21  ...  11.585714  3.620819   4.79  28.08   
1961-05-14  3.782947   4.63  12.33  ...   7.822857  5.460237   6.54  18.66   
1961-05-21  2.468906   5.91  15.96  ...   7.114286  2.216889   6.63  12.00   

                                 loc12                              
                 mean       std    min    max       mean       std  
Date                                                                
1961-01-01  18.500000       NaN  15.04  15.04  15.040000       NaN  
1961-01-08  12.481429  4.349139  10.88  16.46  13.238571  1.773062  
1961-01-15  11.125